In [5]:
import pypsa
import pandas as pd
import time
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

#import cufflinks as cf
#import plotly.offline as pltly
#pltly.init_notebook_mode(connected=True)

In [7]:
network = pypsa.Network()
network.import_from_hdf5("data/networks/euro_50")
network.snapshots = network.snapshots[0:3]

Importing PyPSA from older version of PyPSA than current version 0.17.0.
Please read the release notes at https://pypsa.org/doc/release_notes.html
carefully to prepare your network for import.

INFO:pypsa.io:Imported network euro_50 has buses, carriers, generators, global_constraints, links, loads


In [8]:
network.lopf(network.snapshots, 
            solver_name='gurobi')

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x653
  Lower bound: 16441872722.259996
  Upper bound: 16441872722.259996
  Number of objectives: 1
  Number of constraints: 1070
  Number of variables: 653
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 653
  Number of nonzeros: 2266
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Sta

('ok', 'optimal')

In [10]:
snapshots = network.snapshots

In [24]:
network.generators_t.p.loc[snapshots[0]]

name
AT onwind       0.0
AT solar        0.0
AT ocgt      6938.0
BA onwind       0.0
BA solar        0.0
              ...  
SI solar        0.0
SI ocgt      1146.0
SK onwind       0.0
SK solar        0.0
SK ocgt      3168.0
Name: 2011-01-01 00:00:00, Length: 111, dtype: float64

In [58]:
network.loads_t.p_set.loc[snapshots[0]]

AT     6938.0
BA     1473.0
BE     9964.0
BG     4677.0
CH     6643.0
CZ     6704.0
DE    46209.0
DK     3683.0
EE      923.0
ES    22895.0
FI    10496.0
FR    68546.0
GB    34849.0
GR     4903.0
HR     1962.0
HU     4070.0
IE     3061.0
IT    26553.0
LT     1039.0
LU      497.0
LV      655.0
NL    10684.0
NO    16180.0
PL    13766.0
PT     5328.0
RO     6132.0
RS     6105.0
SE    17565.0
SI     1146.0
SK     3168.0
Name: 2011-01-01 00:00:00, dtype: float64

In [29]:
buses = network.buses.index

In [31]:
load = network.loads_t.p_set.loc[snapshots[0]]['AT']
generation = network.generators_t.p.loc[snapshots[0]]['AT']

generation/load

KeyError: 'AT'

In [33]:
network.generators_t.p.loc[snapshots[0]]

name
AT onwind       0.0
AT solar        0.0
AT ocgt      6938.0
BA onwind       0.0
BA solar        0.0
              ...  
SI solar        0.0
SI ocgt      1146.0
SK onwind       0.0
SK solar        0.0
SK ocgt      3168.0
Name: 2011-01-01 00:00:00, Length: 111, dtype: float64

In [39]:
network.generators.bus == 'AT'

name
AT onwind     True
AT solar      True
AT ocgt       True
BA onwind    False
BA solar     False
             ...  
SI solar     False
SI ocgt      False
SK onwind    False
SK solar     False
SK ocgt      False
Name: bus, Length: 111, dtype: bool

In [44]:
network.generators[network.generators.bus == 'AT'].index

Index(['AT onwind', 'AT solar', 'AT ocgt'], dtype='object', name='name')

In [50]:
network.generators_t.p.loc[:,'AT onwind']

name
2011-01-01 00:00:00    0.0
2011-01-01 01:00:00    0.0
2011-01-01 02:00:00    0.0
2011-01-01 03:00:00    0.0
2011-01-01 04:00:00    0.0
                      ... 
2011-12-31 19:00:00    0.0
2011-12-31 20:00:00    0.0
2011-12-31 21:00:00    0.0
2011-12-31 22:00:00    0.0
2011-12-31 23:00:00    0.0
Freq: H, Name: AT onwind, Length: 8760, dtype: float64

In [84]:
def calc_autoarky(network):
    # calculates the autoarky of a model solution 
    # autoarky is calculated as the mean self sufficiency (energy produced/energy consumed) of all countries in all hours
    mean_autoarky = []
    for timestamp in snapshots:

        hourly_load = network.loads_t.p_set.loc[snapshots[0]]
        hourly_production = {}
        hourly_autoarky = {}
        for bus in buses : 
            generators = network.generators[network.generators.bus == 'AT'].index
            hourly_production[bus] = sum(network.generators_t.p.loc[timestamp,generator] for generator in generators)

            if hourly_production[bus]/hourly_load[bus] > 1 :
                hourly_autoarky[bus] = 1
            else :
                hourly_autoarky[bus] = hourly_production[bus]/hourly_load[bus]
        mean_autoarky.append(np.mean(list(hourly_autoarky.values())))

    return np.mean(mean_autoarky)

In [85]:
calc_autoarky(network)

0.7673773686920043